In [16]:
import pandas as pd
import glob
import re
import math
import urllib.request
# from bs4 import BeautifulSoup
from operator import itemgetter
import networkx as nx
import csv
import os
import matplotlib.pyplot as plt


def native_vote_csv_df():
    filepath = "/Users/tdt62/Desktop/GraduateResearch/test_data/2018_10_17_09_vote*"
    list_ = []

#     filepath = "/projects/canis/nativevote18/twitter/data/2018_10_17_09_vote_stream_1_clean.csv"
#     filepath = "/projects/canis/nativevote18/twitter/data/2018_10_*vote*"

    # Takes all of the csv file and makes one big dataframe
    for name in glob.glob(filepath):
        if(os.stat(name).st_size == 0) == True:
            continue
        else:
            df = pd.read_csv(name,index_col=None, sep='\t')
            list_.append(df)

#     filepath = "/projects/canis/nativevote18/twitter/data/2018_11*stream_1*"
# #     filepath = "/Users/tdt62/Desktop/GraduateResearch/test_data/2018_11_*vote*"

#     # Takes all of the csv file and makes one big dataframe
#     for name in glob.glob(filepath):
#         if(os.stat(name).st_size == 0) == True:
#             continue
#         else:
#             df = pd.read_csv(name,index_col=None, header=0)
#             list_.append(df)

#     filepath = "/projects/canis/nativevote18/twitter/data/2018_12*stream_1*"
# #     filepath = "/Users/tdt62/Desktop/test_data/2018_12_01*vote*"

#         # Takes all of the csv file and makes one big dataframe
#         for name in glob.glob(filepath):
#             if(os.stat(name).st_size == 0) == True:
#                 continue
#             else:
#                 df = pd.read_csv(name,index_col=None, header=0)
#                 list_.append(df)

#         filepath = "/projects/canis/nativevote18/twitter/data/2019_01*stream_1*"

#     # Takes all of the csv file and makes one big dataframe
#     for name in glob.glob(filepath):
#         if(os.stat(name).st_size == 0) == True:
#             continue
#         else:
#             df = pd.read_csv(name,index_col=None, header=0)
#             list_.append(df)

    # Makes the big df in memory
    frame = pd.concat(list_, axis = 0, ignore_index = True)
    frame.fillna("NA", inplace=True)
    return frame

def total_tweets(df):

    # Gets the total number of tweets
    total_tweets_num = df.shape[0]  # gives number of row count
    return total_tweets_num

def unique_tweets(df):

    # Gets the unique tweets i.e. no retweeted status
    df['rt_isdigit'] = list(map(lambda x: str(x).isdigit(), df['Reteeted_Status']))
    unique_df = df.loc[df['rt_isdigit'] == False]
    return unique_df

def unique_users(df):
    unique_users = len(df.User_ID.unique())
    return unique_users

def original_content_user(df):

    # Gets the unique tweets i.e. no retweeted status
    df['rt_isdigit'] = list(map(lambda x: str(x).isdigit(), df['Reteeted_Status']))
    unique_df = df.loc[df['rt_isdigit'] == False]
    unique_users = len(unique_df.User_ID.unique())
    return unique_users

def hashtags(df, top_val):

    top_list = []
    hashtag_dict = {}

    def iterate_hashtags(x):
        hashtag_list = list(x.split("'text':"))
        for element in hashtag_list:
            stripped_element = element.split(',')[0].strip("' '{}[]")
            if stripped_element in hashtag_dict and stripped_element != '[]' and stripped_element != 'NA' and stripped_element != '':
                hashtag_dict[stripped_element] += 1
            else:
                hashtag_dict[stripped_element] = 1

    # Create the hashtag dict
    list(map(lambda x: iterate_hashtags(x), df['Hashtags']))

    # Gets the sorted news_list in descending order
    sorted_news_list = (list(sorted(hashtag_dict.items(), key=itemgetter(1), reverse=True)))

    # Gets the top 20 results of the sorted list
    sorted_news_list = sorted_news_list[0:top_val]

    # Stores all the names in a list, so we can read correctky
    for item in sorted_news_list:
        top_list.append(item[0])

    return top_list, sorted_news_list

def make_domain_csv(top_news_dict):
    with open('news_domains.csv', 'w') as csv_file:
        writer = csv.writer(csv_file)
        for key, value in top_news_dict.items():
            writer.writerow([key, value])

def top_user_mentions(df, top_val):

    top_users_list = []
    hashtag_dict = {}

    def iterate_user_mentions(x):

        hashtag_list = list(x.split("'screen_name':"))
        for element in hashtag_list:
            stripped_element = element.split(',')[0].strip("' '{}[]")
            if stripped_element in hashtag_dict and stripped_element != '[]' and stripped_element != 'NA' and stripped_element != '':
                hashtag_dict[stripped_element] += 1
            else:
                hashtag_dict[stripped_element] = 1

    # Create the hashtag dict
    list(map(lambda x: iterate_user_mentions(x), df['User_Mentions']))

    # Gets the sorted news_list in descending order
    sorted_news_list = (list(sorted(hashtag_dict.items(), key=itemgetter(1), reverse=True)))

    # Gets the top 20 results of the sorted list
    sorted_news_list = sorted_news_list[0:top_val]

    # Stores all the names in a list, so we can read correctky
    for item in sorted_news_list:
        top_users_list.append(item[0])

    return top_users_list, sorted_news_list

def get_hashtag_users_df(df):
        hashtag_users_df = df.loc[(df['User_Mentions'] != '[]') & (df['Hashtags'] != '[]')]
        return hashtag_users_df

def build_mentions_dict(df):

    def make_csv(user_hashtag_dict):
        with open('user_hashtag.csv', 'w') as csv_file:
            writer = csv.writer(csv_file)
            for key, value in user_hashtag_dict.items():
                writer.writerow([key, value])

    def make_lists(df):
        user_list = list(df['User_Mentions'].split("'screen_name':"))
        hashtag_list = list(df['Hashtags'].split("'text':"))



    user_list = list(df['User_Mentions'].split("'screen_name':"))
    hashtag_list = list(df['Hashtags'].split("'text':"))

    for user in user_list:
        stripped_user = user.split(',')[0].strip("' '{}[]")
        if stripped_user != '[]' and stripped_user != 'NA' and stripped_user != '':
            for hashtag in hashtag_list:
                stripped_hashtag = hashtag.split(',')[0].strip("' '{}[]")
                if stripped_hashtag != '[]' and stripped_hashtag != 'NA' and stripped_hashtag != '':
                    dict_tuple = (stripped_user, stripped_hashtag)
                    if(dict_tuple in user_hashtag_dict):
                        user_hashtag_dict[dict_tuple] += 1
                    else:
                        user_hashtag_dict[dict_tuple] = 1
    make_csv(user_hashtag_dict)
    return user_hashtag_dict

In [19]:
native_df = native_vote_csv_df()

unique_df = unique_tweets(native_df)

hashtag_user_df = get_hashtag_users_df(unique_df)

user_hashtag_dict = {}

def build_mentions_dict(df):
    
    def make_csv(user_hashtag_dict):
        with open('user_hashtag.csv', 'w') as csv_file:
            writer = csv.writer(csv_file)
            for key, value in user_hashtag_dict.items():
                writer.writerow([key, value])
            
    
    user_list = list(df['User_Mentions'].split("'screen_name':"))
    hashtag_list = list(df['Hashtags'].split("'text':"))
    print(user_list, '\n\n')
                        
    for user in user_list:
        stripped_user = user.split(',')[0].strip("' '{}[]")
        if stripped_user != '[]' and stripped_user != 'NA' and stripped_user != '':
            for hashtag in hashtag_list:
                stripped_hashtag = hashtag.split(',')[0].strip("' '{}[]")
                if stripped_hashtag != '[]' and stripped_hashtag != 'NA' and stripped_hashtag != '':
                    dict_tuple = (stripped_user, stripped_hashtag)
#                     print(dict_tuple)
                    if(dict_tuple in user_hashtag_dict):
                        user_hashtag_dict[dict_tuple] += 1
                    else:
                        user_hashtag_dict[dict_tuple] = 1
    make_csv(user_hashtag_dict)
    return user_hashtag_dict
                        


            
hashtag_user_df.apply(build_mentions_dict, axis=1)

['User_Mentions'] 


["{'NicolleDWallace': 860555190}"] 


["{'jn4ush': 899371613396893696, 'SamuelWestOK': 1446455844}"] 


['{}'] 


['{}'] 


['{}'] 


['{}'] 


['{}'] 


["{'MickCornett': 18812454, '1AmericanMama': 1031250716243968000}"] 


['{}'] 


['{}'] 


['{}'] 


["{'DinoRossiWA': 146241212}"] 


["{'KristiNoem': 114388293}"] 


["{'SenatorHeitkamp': 1061029050, 'TrahantReports': 28358250, 'StandingRockST': 772941628134699008, 'levinecarrie': 366602165, 'Deb4CongressNM': 286476728, 'SimonMoyaSmith': 89788739, '1a': 793442243734544384}"] 


['{}'] 


['{}'] 


['{}'] 


["{'PauletteEJordan': 2942421722}"] 


["{'PauletteEJordan': 2942421722}"] 


["{'PauletteEJordan': 2942421722}"] 


['{}'] 


['{}'] 


['{}'] 


["{'sharicedavids': 950952511556538368}"] 


['{}'] 


["{'KamalaHarris': 30354991, 'SenWarren': 970207298, 'alanslj': 346108609, 'RepThompson': 303861808, 'LisaServon': 369273877}"] 


['{}'] 


['{}'] 




0      {('User_Mentions', 'Hashtags'): 1, ('NicolleDW...
24     {('User_Mentions', 'Hashtags'): 1, ('NicolleDW...
28     {('User_Mentions', 'Hashtags'): 1, ('NicolleDW...
63     {('User_Mentions', 'Hashtags'): 1, ('NicolleDW...
65     {('User_Mentions', 'Hashtags'): 1, ('NicolleDW...
67     {('User_Mentions', 'Hashtags'): 1, ('NicolleDW...
88     {('User_Mentions', 'Hashtags'): 1, ('NicolleDW...
125    {('User_Mentions', 'Hashtags'): 1, ('NicolleDW...
156    {('User_Mentions', 'Hashtags'): 1, ('NicolleDW...
161    {('User_Mentions', 'Hashtags'): 1, ('NicolleDW...
163    {('User_Mentions', 'Hashtags'): 1, ('NicolleDW...
165    {('User_Mentions', 'Hashtags'): 1, ('NicolleDW...
177    {('User_Mentions', 'Hashtags'): 1, ('NicolleDW...
263    {('User_Mentions', 'Hashtags'): 1, ('NicolleDW...
279    {('User_Mentions', 'Hashtags'): 1, ('NicolleDW...
310    {('User_Mentions', 'Hashtags'): 1, ('NicolleDW...
362    {('User_Mentions', 'Hashtags'): 1, ('NicolleDW...
377    {('User_Mentions', 'Hash

In [65]:
df = pd.read_csv("user_hashtag.csv", names=['User_Hashtag', 'Count'])
df

B = nx.Graph()
# Add nodes with the node attribute "bipartite"
bi1_list = []
bi2_list = []

for val in df['User_Hashtag'][1:]:
    x = val.split(',')
#     print(x[0].strip())
    bi1_list.append(x[0].strip(" ''()"))
    bi2_list.append(x[1].strip(" ''()"))

print(bi1_list)
B.add_nodes_from(set(bi1_list), bipartite=1)
B.add_nodes_from(set(bi2_list), bipartite=0)
B.add_edges_from(list(zip(bi1_list, bi2_list)))

pos = {}

# Update position for node from each group
pos.update((node, (1, index)) for index, node in enumerate(bi2_list))
pos.update((node, (2, index)) for index, node in enumerate(bi1_list))
preds = list(nx.jaccard_coefficient(B, list(zip(bi2_list, bi2_list))))
jaccard_dict = {}
from scipy import stats
rho, pval = stats.spearmanr(bi2_list, bi1_list)
print(rho, pval)
print("--- Jaccard ---")

for n1, n2, j in preds:
    print(n1, n2, j)
    jaccard_dict[(n1, n2)] = j

def make_csv(jaccard_dict):
    with open('jaccard_projection.csv', 'w') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['Usertag, Hashtag', 'JaccardSim'])
        for key, value in user_hashtag_dict.items():
            writer.writerow([key, value])
make_csv(jaccard_dict)
# print(preds)
# print("--- Jaccard ---")
# for n1, n2, j in preds:
#     print("{}, {}: {}".format(n1, n2, j))
# print("--- Jaccard ---\n\n")
# nx.draw(B, pos=pos, with_labels = True)
# nx.bipartite.sets(B, top_nodes=bi1_list)
# G = nx.bipartite.weighted_projected_graph(B, bi1_list)
# nx.draw(G, with_labels = True)
# plt.show()

['"NicolleDWallace\': 860555190"', '"jn4ush\': 899371613396893696"', '"MickCornett\': 18812454"', '"DinoRossiWA\': 146241212"', '"KristiNoem\': 114388293"', '"SenatorHeitkamp\': 1061029050"', '"PauletteEJordan\': 2942421722"', '"sharicedavids\': 950952511556538368"', '"KamalaHarris\': 30354991"']
0.11666666666666665 0.7650079429261462
--- Jaccard ---
VoteMeTooPAC VoteMeTooPAC 1.0
powertweet powertweet 1.0
GOP GOP 1.0
Ballots Ballots 1.0
MAGA MAGA 1.0
NoDAPL NoDAPL 1.0
imwithpaulette imwithpaulette 1.0
ElectWomen ElectWomen 1.0
studentloans studentloans 1.0


In [1]:
import community
import networkx as nx
import matplotlib.pyplot as plt

#better with karate_graph() as defined in networkx example.
#erdos renyi don't have true community structure
G = nx.erdos_renyi_graph(30, 0.05)

#first compute the best partition
partition = community.best_partition(G)

#drawing
size = float(len(set(partition.values())))
pos = nx.spring_layout(G)
count = 0.
for com in set(partition.values()) :
    count = count + 1.
    list_nodes = [nodes for nodes in partition.keys()
                                if partition[nodes] == com]
    nx.draw_networkx_nodes(G, pos, list_nodes, node_size = 20,
                                node_color = str(count / size))


nx.draw_networkx_edges(G,pos, alpha=0.5)
plt.show()

ModuleNotFoundError: No module named 'community'